In [1]:
import scipy.io
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import math
import soundfile as sf
import os
from pydub import AudioSegment
from tqdm import tqdm
import random
random.seed(4)


In [2]:
# prendi il nome di tutti i files
audio_path = "D:\\Giacomo\\Tovanella-20241110T120546Z-001\\Tovanella"
audio_names = os.listdir(audio_path)
audio_names[:5]

['20190601_000000.WAV',
 '20190601_010000.WAV',
 '20190601_020000.WAV',
 '20190601_030000.WAV',
 '20190601_040000.WAV']

In [3]:
import json

category_info = {}
audio_info = {}
with open('utils\coords.json') as f:
    category_info = json.load(f)
with open('utils\inverted_coords.json') as f:
    audio_info = json.load(f)


In [4]:
des_segm_path = "D:\\Giacomo\\Tovanella-20241110T120546Z-001\\segments_2"
os.makedirs(des_segm_path, exist_ok=True)
os.makedirs(os.path.join(des_segm_path, "train"), exist_ok=True)
os.makedirs(os.path.join(des_segm_path, "test"), exist_ok=True)

In [5]:
# per ogni categoria, prende tutti i file associati
# per ogni file, ne prende la durata
# se troppo corto (<3s) viene ignorato
# altrimenti viene salvato il time_frame da cui estrarre il segmento da 3s
# ogni segmento viene salvato sotto la categoria a cui appartiene
# tutti gli audio della stessa categoria vengono mescolati

target_dir = "D:\\Giacomo\\Tovanella-20241110T120546Z-001\\Tovanella"
extracted_info = {}
for category in category_info:
    files_info = category_info[category]
    extracted_info[category] = []
    for file in files_info:
        duration = file["duration"]
        if file["file_name"] not in audio_names or duration < 3:
            continue
        segments_start_time = [ file["start_time"] + i*3 for i in range(duration // 3) ]
        full_path = os.path.join(target_dir, file["file_name"])
        for segment in segments_start_time:
            extracted_info[category].append({"file_name": full_path, "start_time": segment})
    random.shuffle(extracted_info[category])

In [6]:
# impiega 7 minuti
categories = list(extracted_info.keys())
for i in range(len(categories)):
    category = categories[i]
    os.makedirs(os.path.join(des_segm_path, "train", category), exist_ok=True)
    os.makedirs(os.path.join(des_segm_path, "test", category), exist_ok=True)
    all_category_audio = extracted_info[category]
    total_length = len(all_category_audio)
    threshold = 0.2
    train_length = int(total_length * 0.7)
    test_length = total_length - train_length
    print(f"Loading {category} category... {i}/{len(categories)}")
    for i in tqdm(range(total_length)):
        split = "train" if i < train_length else "test"
        full_path = all_category_audio[i]["file_name"]
        file_name = full_path.split("\\")[-1]
        file_name = file_name[:-4]
        export_path = os.path.join(des_segm_path, split, category, file_name+f"_{i}"+".wav")
        if os.path.exists(export_path):
            continue
        audio = AudioSegment.from_file(full_path, format="wav")
        start_time = int(all_category_audio[i]["start_time"])
        segment = audio[start_time:start_time + 3000]
        segment.export(export_path, format="wav")


Loading Wind_ category... 0/26


100%|██████████| 126/126 [00:10<00:00, 12.20it/s]


Loading Regulus ignicapilla_Common Firecrest category... 1/26


100%|██████████| 296/296 [00:22<00:00, 13.32it/s]


Loading Sylvia atricapilla_Eurasian Blackcap category... 2/26


...

100%|██████████| 40/40 [00:04<00:00,  8.50it/s]


Loading Dryocopus martius_Black Woodpecker category... 24/26


100%|██████████| 15/15 [00:02<00:00,  7.18it/s]


Loading Phylloscopus trochilus_Willow Warbler category... 25/26


100%|██████████| 1/1 [00:00<00:00,  7.92it/s]
